<a href="https://colab.research.google.com/github/pkse-searcher/fsvm-pois-attack-defense/blob/main/XFed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import svm
import random
from random import randint, shuffle, uniform
from copy import copy,deepcopy
from datetime import datetime
import numpy as np
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
import itertools

from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from math import isclose, fabs, sqrt

from joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

import pip

def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])

try:
  import secml
except ImportError:
  %pip install git+https://github.com/pralab/secml
from secml.ml.classifiers import CClassifierSVM
from secml.ml.kernels import CKernelRBF
from secml.adv.attacks import CAttackPoisoningSVM
from secml.data import CDataset
from secml.data.splitter import CTrainTestSplit
from secml.ml.kernels.c_kernel_linear import CKernelLinear
from secml.data.loader import CDLRandomBlobs
from secml.array import CArray
from sklearn.model_selection import train_test_split
# NBVAL_IGNORE_OUTPUT
from secml.data.loader import CDataLoaderMNIST, CDataLoaderCIFAR10


import gzip
import struct
from array import array
from multiprocessing import Lock
import numpy as np

from secml.data.loader import CDataLoader
from secml.data import CDataset, CDatasetHeader
from secml.array import CArray
from secml.utils import fm
from secml.utils.download_utils import dl_file_gitlab, md5
from secml.settings import SECML_DS_DIR




try:
  from pyod.models.knn import KNN
except:
  install('pyod')
  from pyod.models.knn import KNN
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.copod import COPOD
import matplotlib.pyplot as plt

import bz2
from sklearn.datasets import load_svmlight_file
import pandas as pd

import sqlite3
from sqlalchemy import create_engine, inspect



In [ ]:
!ls -l /content/results/experiments.db
!cat /content/results/experiments.db

-rw-r--r-- 1 root root 2756608 Jun  4 19:15 /content/results/experiments.db


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



@��8��0
�
�
!AdMNISTPERCEPTRONBEAT2FEDINDIVKRUM22=?㮬#�س2024-06-01 12:52:43.197605T��
@��8��0
�
�
(	�	x	 �p� �i�b
!AdMNISTPERCEPTRONBEAT2FEDINDIVKRUM22
!AdMNISTPERCEPTRONBEAT2FEDINDIVKRUM22?��KF
!AdMNISTPERCEPTRONBEAT2FEDINDIVKRUM22Y?�uxd�
@��8��0
�
�
!AdMNISTPERCEPTRONBEAT2FEDINDIVKRUM22<?�
߽�
@��8��0
�
�
!AdMNISTPERCEPTRONBEAT2FEDINDIVKRUM22Y?���yJ1�2024-06-01 12:53:17.243359T��
!AdMNISTPERCEPTRONBEAT2FEDINDIVKRUM22M?�
@��8��0
�
�
!AdMNISTPERCEPTRONBEAT2FEDINDIVKRUM22
@��8��0
�
�
B��:��2
�
�
*	�	z	"�r�j�b
!Ad#MNISTPERCEPTRONBEAT2FEDINDIVKRUM22?��6�2024-06-01 12:53:52.033773T��
!Ad#MNISTPERCEPTRONBEAT2FEDINDIVKRUM22
@��8��0
�
�
@� ��:��2
�
�
*	�	z	"�r�j�b
!Ad(MNISTPERCEPTRONBEAT2FEDINDIVKRUM22
@��8��0
�
�
!Ad(MNISTPERCEPTRONBEAT2FEDINDIVKRUM22-?��6�2024-06-0

In [ ]:
# Create an SQLite database connection
engine = create_engine('sqlite:////content/results/experiments.db')

# Function to store results
def store_results(params, accuracy):
    df = pd.DataFrame([params])
    df['accuracy'] = accuracy
    df['timestamp'] = datetime.now()

    # Check if the table exists and create if not
    inspector = inspect(engine)
    if not inspector.has_table('experiment_results'):
        df.to_sql('experiment_results', con=engine, if_exists='replace', index=False)
    else:
        # Check if the parameters already exist
        with engine.connect() as conn:
            condition = " AND ".join([f"{key} = {value}" if isinstance(value, (int, float)) else f"{key} = '{value}'" for key, value in params.items()])
            query = f"SELECT * FROM experiment_results WHERE {condition}"
            existing_rows = pd.read_sql(query, con=conn)

            if not existing_rows.empty:
                # Update existing row
                update_query = f"UPDATE experiment_results SET accuracy = {accuracy}, timestamp = '{datetime.now()}' WHERE {condition}"
                conn.execute(update_query)
            else:
                # Insert new row
                df.to_sql('experiment_results', con=conn, if_exists='append', index=False)


In [ ]:

def dataLoad(dataset_type, p = 50):
    if dataset_type == 'MNIST':
        # MNIST dataset will be downloaded and cached if needed
        loader = CDataLoaderMNIST()

        slc = 100
        size_all = 784

        # total MNIST data with 5 and 9 is 11370
        n_samples = slc * num_local_machine  # Number of samples
        digits = (9, 8)

        dataset = loader.load('training', digits=digits)
        # Normalize the features in `[0, 1]`
        dataset.X /= 255
        XGlobal, yGlobal = dataset.X.get_data(), dataset.Y.get_data()

        test_dataset = loader.load('testing', digits=digits, num_samples=1000)
        test_dataset.X /= 255
        testX, testy = test_dataset.X.get_data(), test_dataset.Y.get_data()


    elif dataset_type == 'IJCNN1':

        XGlobal, yGlobal = load_svmlight_file('/content/sample_data/ijcnn1.tr.bz2')
        XGlobal = XGlobal.toarray()
        yGlobal = [0 if y < 0 else 1 for y in yGlobal]

        testX, testy = load_svmlight_file('/content/sample_data/ijcnn1.t.bz2')
        testX = testX.toarray()
        testy = [0 if y < 0 else 1 for y in testy]

        slc = 300


    elif dataset_type == 'ACOUSTIC':

        XGlobal, yGlobal = load_svmlight_file('/content/sample_data/acoustic_scale.bz2')
        XGlobal = XGlobal.toarray()
        # print(yGlobal)
        yGlobal = [0 if y < 2 else 1 for y in yGlobal]
        # print(yGlobal)

        testX, testy = load_svmlight_file('/content/sample_data/acoustic_scale.t.bz2')
        testX = testX.toarray()
        testy = [0 if y < 2 else 1 for y in testy]

        slc = 300


    elif dataset_type == 'madelon':

        XGlobal, yGlobal = load_svmlight_file('/content/sample_data/madelon')
        XGlobal = XGlobal.toarray()
        # print(yGlobal)
        yGlobal = [0 if y < 0 else 1 for y in yGlobal]
        # print(yGlobal)

        # Shuffle the dataset
        XGlobal, yGlobal = shuffle(XGlobal, yGlobal, random_state=42)

        testX, testy = load_svmlight_file('/content/sample_data/madelon.t')
        testX = testX.toarray()
        testy = [0 if y < 0 else 1 for y in testy]

        slc = 20


    elif dataset_type == 'FASHION_MNIST':
        # MNIST dataset will be downloaded and cached if needed
        train_df = pd.read_csv('/content/sample_data/fashion-mnist_train.csv')
        test_df = pd.read_csv('/content/sample_data/fashion-mnist_test.csv')
        print('The shape of training dataset : ', train_df.shape)
        # print('The shape of testing dataset : ', test_df.shape)

        train = np.array(train_df, dtype = 'float32')
        test = np.array(test_df, dtype = 'float32')


        trainX = train[:,1:]/255

        trainY = train[:,0]

        dataX = test[:,1:]/255

        dataY=test[:,0]

        def gen_data(X, Y):
            retX, retY = [], []
            for i in range(len(Y)):
                # print(i, trainY[i])
                lbl = int(Y[i])
                if lbl == 0 or lbl == 6:
                    retX.append(X[i])
                    retY.append(0 if lbl == 0 else 1)

            return retX, retY

        XGlobal, yGlobal = gen_data(trainX, trainY)
        unique, counts = np.unique(yGlobal, return_counts=True)
        print(f"Global count {dict(zip(unique, counts))}")
        testX, testy = gen_data(dataX, dataY)

        # print(len(trainY), len(dataY))

        unique, counts = np.unique(testy, return_counts=True)
        print(f"Test count {dict(zip(unique, counts))}")
        # print(len(yGlobal), testy)

        slc = 100
        size_all = 784


    elif dataset_type == 'CIFAR10':
        # CIFAR10 dataset will be downloaded and cached if needed
        loader = CDataLoaderCIFAR10()

        def rgb_2_gray(arr):
            return [0.2989 * arr[i] + 0.5870 * arr[i+1024] + 0.1140 * arr[i+2048] for i in range(1024)]

        size_all = 1024
        slc = 100

        # total MNIST data with 5 and 9 is 11370
        n_samples = slc * num_local_machine  # Number of samples

        dataset, test_dataset = loader.load()
        # Normalize the features in `[0, 1]`
        dataset.X /= 255

        XGlobal, yGlobal = [], []
        trainX = dataset.X.get_data()
        trainY = dataset.Y.get_data()

        for i in range(len(trainY)):
            if trainY[i] < 2:
                XGlobal.append(rgb_2_gray(trainX[i]))
                yGlobal.append(trainY[i])

        unique, counts = np.unique(yGlobal, return_counts=True)
        print(f"Global count {dict(zip(unique, counts))}")
        testX, testy = [], []
        test_dataset.X /= 255
        dataX = test_dataset.X.get_data()
        dataY = test_dataset.Y.get_data()

        print(len(trainY), len(dataY))

        for i in range(len(dataY)):
            if dataY[i] < 2:
                testX.append(rgb_2_gray(dataX[i]))
                testy.append(dataY[i])

        unique, counts = np.unique(testy, return_counts=True)
        print(f"Test count {dict(zip(unique, counts))}")
        print(len(yGlobal), len(testy))
    else:
        raise ValueError("Invalid dataset type")

    if p == -1:
        data = [[], []]
        sz = len(yGlobal)
        for i in range(sz):
            data[yGlobal[i]].append(XGlobal[i])
        XGlobal, yGlobal = [], []
        print(f"Changing data, p={p} and slc = {slc}")
        for edge_device in range(num_local_machine):
            label = 0
            data_iid = edge_device + (0 if edge_device >= (num_local_machine//2) else 1)
            print(f"edge_device {edge_device}, 0: {len(data[0])}, 1: {len(data[1])}, data_iid {data_iid}")
            for _ in range(data_iid):
                XGlobal.append(data[label].pop())
                yGlobal.append(label)

            anti_label = 1
            for _ in range(slc - data_iid):
                XGlobal.append(data[anti_label].pop())
                yGlobal.append(anti_label)
    elif p != 50:
        data = [[], []]
        sz = len(yGlobal)
        for i in range(sz):
            data[yGlobal[i]].append(XGlobal[i])
        XGlobal, yGlobal = [], []
        data_iid = int((slc * p) / 100 + 0.5)
        print(f"Changing data, p={p} and data_iid = {data_iid} and slc = {slc}")
        for edge_device in range(num_local_machine):
            label = edge_device%2
            for _ in range(slc - data_iid):
                XGlobal.append(data[label].pop())
                yGlobal.append(label)

            anti_label = ( edge_device+1 ) % 2
            for _ in range(data_iid):
                XGlobal.append(data[anti_label].pop())
                yGlobal.append(anti_label)

    return XGlobal, yGlobal, testX, testy, slc



In [ ]:
def krum(W_1, B):
        # Krum
    hSets = deepcopy(W_1)

    krum_scores = []
    byzantine_client_num = num_local_machine // 5
    krum_limit = num_local_machine - byzantine_client_num - 2
    for i in range(0, num_local_machine):
        dists = []
        for j in range(0, num_local_machine):
            if i != j:
                temp_vector = np.subtract(hSets[i], hSets[j])
                distance = np.dot(temp_vector,temp_vector)
                dists.append(distance)
        #   print(i, "all_socre", dists)
        dists.sort()  # ascending
        # print(i, "all_socre", dists[0], dists[-1])
        score = dists[0 : krum_limit]
        # print(i, "low_socre", score[-1])
        krum_scores.append(sum(score))
        #   print(i, sum(score))
    score_index = np.argsort(krum_scores)  # indices; ascending
    score_index = score_index[0 : krum_limit]
    print(krum_scores)
    print(score_index)
    print(f"Dist min {krum_scores[score_index[0]]/krum_limit} and max {krum_scores[score_index[-1]]/krum_limit}")
    hSets = [W_1[i] for i in score_index]

    w = hSets[0]
    b = np.mean(B)

    cnt = 0
    for i in range(len(testX)):
        det = np.dot(testX[i], w) + b
        if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
            cnt += 1
    print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} after krum')
    return w, b



def multi_krum(W_1, B):
        # Krum
    hSets = deepcopy(W_1)
    num_local_machine = len(W_1)

    krum_scores = []
    byzantine_client_num = num_local_machine // 5
    krum_limit = num_local_machine - byzantine_client_num - 2
    for i in range(0, num_local_machine):
        dists = []
        for j in range(0, num_local_machine):
            if i != j:
                temp_vector = np.subtract(hSets[i], hSets[j])
                distance = np.dot(temp_vector,temp_vector)
                dists.append(distance)
        #   print(i, "all_socre", dists)
        dists.sort()  # ascending
        # print(i, "all_socre", dists[0], dists[-1])
        score = dists[0 : krum_limit]
        # print(i, "low_socre", score[-1])
        krum_scores.append(sum(score))
        #   print(i, sum(score))
    score_index = np.argsort(krum_scores)  # indices; ascending
    score_index = score_index[0 : krum_limit]
    # print(krum_scores)
    # print(score_index)
    # print(f"Dist min {krum_scores[score_index[0]]/krum_limit} and max {krum_scores[score_index[-1]]/krum_limit}")
    print(f'Out of {len(score_index)}, {np.sum(score_index < advNumber)} are adversarial')
    hSets = [W_1[i] for i in score_index]

    b = np.mean(B)
    w = np.mean(hSets, axis=0)

    cnt = 0
    for i in range(len(testX)):
        det = np.dot(testX[i], w) + b
        if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
            cnt += 1
    print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} after multi-krum')
    return w, b


In [ ]:

#for fed-avg
# Assuming dataLoad() function initializes XGlobal, yGlobal, testX, testy, slc
# dataLoad()  # Ensure this function is defined

num_local_machine = 100
advNumber = 30
dataset_type = 'ACOUSTIC'
MACHINE_LEARNING_ALGO = 'PERCEPTRON' # 'SVM' #'LR'
ATTACK_TYPE='BEAT2FED'
MODE='INDIV'
AGGREGATION = 'MEAN' # 'KRUM'
iid_deg=50
poisoning_percent=100

for MACHINE_LEARNING_ALGO, dataset_type, ATTACK_TYPE, MODE, AGGREGATION, iid_deg, poisoning_percent, advNumber in\
 itertools.product(['SVM', 'LR', 'PERCEPTRON'], ['madelon'], ['BEAT2FED'],\
                   ['INDIV'], ['MEAN', 'KRUM'], [50], [50],\
                   [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]) :
# for MACHINE_LEARNING_ALGO, dataset_type, ATTACK_TYPE, MODE, AGGREGATION, iid_deg, poisoning_percent, advNumber in\
#  itertools.product(['SVM', 'LR', 'PERCEPTRON'], ['FASHION_MNIST', 'MNIST', 'ACOUSTIC', 'IJCNN1', 'CIFAR10'], ['BEAT2FED'],\
#                    ['INDIV'], ['MEAN', 'KRUM'], [50], [50],\
#                    [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]) :
# for MACHINE_LEARNING_ALGO, dataset_type, ATTACK_TYPE, MODE, AGGREGATION, iid_deg, poisoning_percent, advNumber in\
#  itertools.product(['SVM'], ['CIFAR10'], ['BEAT2FED'],\
#                    ['INDIV'], ['MEAN'], [50], [50],\
#                    [20]) :

    print()
    print(f"Time {datetime.now()}:", MACHINE_LEARNING_ALGO, dataset_type, ATTACK_TYPE, advNumber, MODE, AGGREGATION, iid_deg, poisoning_percent)
    XGlobal, yGlobal, testX, testy, slc = dataLoad(dataset_type, p = iid_deg)


    # Function to evaluate accuracy
    def evaluate_accuracy(tW, tb):
        cnt = 0
        for i in range(len(testX)):
            det = np.dot(testX[i], tW) + tb
            if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
                cnt += 1
        accuracy = cnt / len(testX)

        params = {
            'num_local_machine': num_local_machine,
            'advNumber': advNumber,
            'dataset_type': dataset_type,
            'MACHINE_LEARNING_ALGO': MACHINE_LEARNING_ALGO,
            'ATTACK_TYPE': ATTACK_TYPE,
            'MODE': MODE,
            'AGGREGATION': AGGREGATION,
            'iid_deg': iid_deg,
            'poisoning_percent': poisoning_percent,
            'round': rnd
        }
        store_results(params, accuracy)

        return accuracy

    def average_euclidean_distance(global_models):
        num_models = len(global_models)
        # mu = 0

        # # Compute pairwise distances
        # for i in range(num_models):
        #     for j in range(i + 1, num_models):
        #         mu = max(mu, np.linalg.norm(global_models[i] - global_models[j]))

        # Calculate the average distance

        # Convert list of ndarrays to a 2D array for pdist
        global_models_array = np.vstack(global_models)

        # Compute pairwise Euclidean distances and calculate the standard deviation
        pairwise_distances = pdist(global_models_array, metric='euclidean')
        mu = 3*np.std(pairwise_distances)

        return mu


    def multi_krum(W_1, B):
        # Krum
        hSets = deepcopy(W_1)
        num_local_machine = len(W_1)

        krum_scores = []
        byzantine_client_num = num_local_machine // 5
        krum_limit = num_local_machine - byzantine_client_num - 2

        for i in range(num_local_machine):
            dists = []
            for j in range(num_local_machine):
                if i != j:
                    temp_vector = np.subtract(hSets[i], hSets[j])
                    distance = np.dot(temp_vector, temp_vector)
                    dists.append(distance)
            dists.sort()  # ascending
            score = dists[:krum_limit]
            krum_scores.append(sum(score))

        score_index = np.argsort(krum_scores)[:krum_limit]  # indices; ascending

        print(f'Out of {len(score_index)}, {np.sum(score_index < advNumber)} are adversarial')

        hSets = [W_1[i] for i in score_index]
        b = np.mean(B)
        w = np.mean(hSets, axis=0)

        return w, b


    def aggregation(W, B):
        if AGGREGATION == 'MEAN':
            return np.mean(W, axis=0), np.mean(B)
        elif AGGREGATION == 'KRUM':
            return multi_krum(W, B)
        else:
            raise ValueError("Invalid aggregation method")



    # Initialize lists to store weights and biases
    global_models = []

    global_w, global_b = np.zeros(len(XGlobal[0])), 0
    prev_global_w = global_w

    # Store initial global model
    # global_models.append(global_w)


    initial_accuracy = evaluate_accuracy(global_w, global_b)
    print(f'Time {datetime.now()}: Accuracy is {initial_accuracy} before attack')

    # Multi-round federated learning with attack
    alpha = 1.1  # Scaling factor for increasing mu
    beta = 0.9  # Scaling factor for decreasing mu

    for rnd in range(100):
        W_1, B_1 = [], []

        for local_machine in range(num_local_machine):
            Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
            ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])
            # sgd = SGDClassifier(loss='log', max_iter=100, tol=1e-3, learning_rate='constant', eta0=0.01)
            if MACHINE_LEARNING_ALGO == 'SVM':
                C = 1
                alpha = 1 / (len(ylocal) * C)
                # SGD Classifier configured as SVM
                sgd = SGDClassifier(loss='hinge', alpha=alpha, max_iter=1000, tol=1e-3, random_state=42)
            elif MACHINE_LEARNING_ALGO == 'LR':
                sgd = SGDClassifier(loss='log', max_iter=20 * len(ylocal), tol=1e-4, learning_rate='optimal', random_state=42)
            elif MACHINE_LEARNING_ALGO == 'PERCEPTRON':
                sgd = SGDClassifier(loss='perceptron', max_iter=1000, tol=1e-3, random_state=42)
            # elif MACHINE_LEARNING_ALGO == 'LDA':
            #     # Linear Discriminant Analysis equivalent
            #     sgd = LinearDiscriminantAnalysis()
            else:
                raise ValueError("Invalid machine learning algorithm")




            sgd.coef_ = np.array([deepcopy(global_w)])  # Set the modified coefficients
            sgd.intercept_ = np.array([deepcopy(global_b)])  # Set the modified intercept
            sgd.partial_fit(Xlocal, ylocal, classes=np.unique(ylocal))

            b = deepcopy(sgd.intercept_[0])  # Intercept
            w = deepcopy(sgd.coef_[0])  # Coefficients

            W_1.append(w)
            B_1.append(b)
            # Step 2: Make predictions on the test data
            predictions = sgd.predict(testX)

            # # Step 3: Calculate the accuracy
            # accuracy = accuracy_score(testy, predictions)
            # print(f'For local machine, {local_machine}, Original Accuracy is {accuracy}')


        if MODE == 'INDIV':
            for local_machine in range(advNumber):
                Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
                ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])

                if rnd == 0:
                    w = W_1[local_machine]
                    b = B_1[local_machine]
                else:
                    w = global_w
                    b = global_b

                v = np.zeros(len(Xlocal[0]))
                for j in range(len(Xlocal)):
                    v += np.array(Xlocal[j]) * (-1 if ylocal[j] == 0 else 1)

                # # Min max attack
                # for indx in range(len(v)):
                #     w[indx] = -fabs(w[indx]) if v[indx] > 0 else fabs(w[indx])
                    # w[indx] = -1000 if v[indx] > 0 else 1000

                unit_vector = v/np.sqrt(np.dot(v,v))
                # Calculate standard deviation of stored global models
                if len(global_models) > 1:
                    mu = max(np.linalg.norm(W_1[local_machine]-global_w), average_euclidean_distance(global_models))
                elif len(global_models) == 1:
                    mu = np.linalg.norm(W_1[local_machine]-global_w)
                else:
                    # mu = np.linalg.norm(W_1[local_machine]-global_w)
                    #mu = 0.1  # Initial perturbation magnitude if only one global model exists
                    mu = 1
                # print(f"For round {rnd} and advNumber {local_machine} mu is {mu}", end=" ")
                delta = mu * unit_vector



                # accuracy = evaluate_accuracy(w, global_b)
                # print(f'Non attacker: Accuracy is {accuracy}', end=" ")

                # # Apply perturbation to the local model
                w = global_w - delta
                # w = w - delta
                # b = b
                # accuracy = evaluate_accuracy(w, b)
                # print(f'Attacker: Accuracy is {accuracy}')
                W_1[local_machine] = w
                B_1[local_machine] = b

            # for local_machine in range(num_local_machine):
            #     accuracy = evaluate_accuracy(W_1[local_machine], B_1[local_machine])
            #     print(f'For local machine {local_machine}, Accuracy is {accuracy}')
        elif advNumber > 0:
            Xlocal = deepcopy(XGlobal[0 : advNumber * slc])
            ylocal = deepcopy(yGlobal[0 : advNumber * slc])

            if rnd == 0:
                w = np.mean(W_1[0 : advNumber], axis=0)
                b = np.mean(B_1[0 : advNumber])
            else:
                w = global_w
                b = global_b

            v = np.zeros(len(Xlocal[0]))
            for j in range(len(Xlocal)):
                v += np.array(Xlocal[j]) * (-1 if ylocal[j] == 0 else 1)

            # # Min max attack
            # for indx in range(len(v)):
            #     w[indx] = -fabs(w[indx]) if v[indx] > 0 else fabs(w[indx])
                # w[indx] = -1000 if v[indx] > 0 else 1000

            unit_vector = v/np.sqrt(np.dot(v,v))

                # Calculate standard deviation of stored global models
                if len(global_models) > 1:
                    mu = max(np.linalg.norm(W_1[local_machine]-global_w), average_euclidean_distance(global_models))
                elif len(global_models) == 1:
                    mu = np.linalg.norm(W_1[local_machine]-global_w)
                else:
                    # mu = np.linalg.norm(W_1[local_machine]-global_w)
                    #mu = 0.1  # Initial perturbation magnitude if only one global model exists
                    mu = 1
                # print(f"For round {rnd} and advNumber {local_machine} mu is {mu}", end=" ")
                delta = mu * unit_vector



                # accuracy = evaluate_accuracy(w, global_b)
                # print(f'Non attacker: Accuracy is {accuracy}', end=" ")

                # # Apply perturbation to the local model
                w = global_w - delta
                # w = w - delta
                # b = b
                # accuracy = evaluate_accuracy(w, b)
                # print(f'Attacker: Accuracy is {accuracy}')
                W_1[local_machine] = w
                B_1[local_machine] = b


        # Update global model
        prev_global_w = global_w
        global_w , global_b = aggregation(W_1, B_1)

        # Store the new global model
        global_models.append(global_w)

        # Evaluate accuracy after attack
        accuracy = evaluate_accuracy(global_w, global_b)
        print(f'Time {datetime.now()}: Accuracy is {accuracy} at round {rnd} after attack')




Streaming output truncated to the last 5000 lines.
Time 2024-06-06 13:52:57.237127: Accuracy is 0.8904592098232298 at round 83 after attack
Out of 78, 0 are adversarial
Time 2024-06-06 13:52:57.632529: Accuracy is 0.8917132855694049 at round 84 after attack
Out of 78, 0 are adversarial
Time 2024-06-06 13:52:58.032779: Accuracy is 0.8917787156083358 at round 85 after attack
Out of 78, 0 are adversarial
Time 2024-06-06 13:52:58.425834: Accuracy is 0.8910698901865847 at round 86 after attack
Out of 78, 0 are adversarial
Time 2024-06-06 13:52:58.824586: Accuracy is 0.892694736153368 at round 87 after attack
Out of 78, 0 are adversarial
Time 2024-06-06 13:52:59.222961: Accuracy is 0.8892814691224741 at round 88 after attack
Out of 78, 0 are adversarial
Time 2024-06-06 13:52:59.622879: Accuracy is 0.8913425153487966 at round 89 after attack
Out of 78, 0 are adversarial
Time 2024-06-06 13:53:00.018618: Accuracy is 0.8897394793949902 at round 90 after attack
Out of 78, 0 are adversarial
Time 2

In [ ]:
# Querying the database
query = "SELECT * FROM experiment_results where round = 99"
results = pd.read_sql(query, con=engine)
print(results)

In [ ]:
# Querying the database
query = "SELECT advNumber,dataset_type,MACHINE_LEARNING_ALGO,ATTACK_TYPE,MODE,AGGREGATION, min(accuracy) FROM experiment_results where round >= 97 \
    group by advNumber,dataset_type,MACHINE_LEARNING_ALGO,ATTACK_TYPE,MODE,AGGREGATION,iid_deg,poisoning_percent\
    order by dataset_type,MACHINE_LEARNING_ALGO,ATTACK_TYPE,MODE,AGGREGATION,advNumber"
results = pd.read_sql(query, con=engine)
print(results.to_string(index=False))

 advNumber  dataset_type MACHINE_LEARNING_ALGO ATTACK_TYPE  MODE AGGREGATION  min(accuracy)
         0      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM       0.808018
         5      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM       0.806851
        10      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM       0.791271
        15      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM       0.756661
        20      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM       0.646536
        25      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM       0.571479
        30      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM       0.233037
        35      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM       0.232428
        40      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM       0.232428
        45      ACOUSTIC                    LR    BEAT2FED INDIV        KRUM    

In [ ]:

#for krum
# Assuming dataLoad() function initializes XGlobal, yGlobal, testX, testy, slc
# dataLoad()  # Ensure this function is defined

num_local_machine = 100
advNumber = 30
dataset_type = 'FASHION_MNIST'
MACHINE_LEARNING_ALGO = 'LR'
dataLoad()
# Initialize lists to store weights and biases
W, B = [], []
global_models = []

# Train local models initially
for local_machine in range(num_local_machine):
    Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
    ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])

    if MACHINE_LEARNING_ALGO == 'LR':
        model = LogisticRegression(max_iter=20*len(ylocal))
        model.fit(Xlocal, ylocal)
        w = model.coef_[0]
        b = model.intercept_[0]

    elif MACHINE_LEARNING_ALGO == 'SVM':
        clfLocals = CClassifierSVM(C=1)
        clfLocals.fit(Xlocal, ylocal)
        w, b = clfLocals.w.get_data()[0], clfLocals.b.get_data()[0]

    elif MACHINE_LEARNING_ALGO == 'PERCEPTRON':
        model = Perceptron()
        model.fit(Xlocal, ylocal)
        w = model.coef_[0]
        b = model.intercept_[0]

    elif MACHINE_LEARNING_ALGO == 'LDA':
        model = LinearDiscriminantAnalysis()
        model.fit(Xlocal, ylocal)
        w = model.coef_[0]
        b = model.intercept_[0]

    W.append(w)
    B.append(b)

# Initialize global model
global_w, global_b = krum(W,B)
prev_global_w = global_w

# Store initial global model
global_models.append(global_w)

# Function to evaluate accuracy
def evaluate_accuracy(global_w, global_b):
    cnt = 0
    for i in range(len(testX)):
        det = np.dot(testX[i], global_w) + global_b
        if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
            cnt += 1
    accuracy = cnt / len(testX)
    return accuracy

initial_accuracy = evaluate_accuracy(global_w, global_b)
print(f'Time {datetime.now()}: Accuracy is {initial_accuracy} before attack')

# Multi-round federated learning with attack
alpha = 1.1  # Scaling factor for increasing mu
beta = 0.9  # Scaling factor for decreasing mu

for round in range(100):
    W_1, B_1 = [], []
    for local_machine in range(num_local_machine):
        Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
        ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])
        sgd = SGDClassifier(loss='log', max_iter=100, tol=1e-3, learning_rate='constant', eta0=0.01)
        sgd.coef_ = np.array([deepcopy(global_w)])  # Set the modified coefficients
        sgd.intercept_ = np.array([deepcopy(global_b)])  # Set the modified intercept
        sgd.partial_fit(Xlocal, ylocal, classes=np.unique(ylocal))

        b = deepcopy(sgd.intercept_[0])  # Intercept
        w = deepcopy(sgd.coef_[0])  # Coefficients

        W_1.append(w)
        B_1.append(b)


    for local_machine in range(advNumber):
        Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
        ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])

        v = np.zeros(len(Xlocal[0]))
        for j in range(len(Xlocal)):
            v = v + np.array(Xlocal[j]) * (-1 if ylocal[j] == 0 else 1)
        unit_vector = v/np.sqrt(np.dot(v,v))
        # Calculate standard deviation of stored global models
        if len(global_models) > 1:
            global_models_array = np.stack(global_models)
            global_std = np.std(global_models_array, axis=0)
            mu = np.mean(global_std)
        else:
            #mu = 0.1  # Initial perturbation magnitude if only one global model exists
            mu= np.linalg.norm(w-global_w)
            delta = mu * unit_vector

        # Apply perturbation to the local model
        w = w + delta
        b = b
        W_1[local_machine] = w
        B_1[local_machine] = b

    # Update global model
    prev_global_w = global_w
    global_w, global_b = krum(W_1,B_1)

    # Store the new global model
    global_models.append(global_w)

    # Evaluate accuracy after attack
    accuracy = evaluate_accuracy(global_w, global_b)
    print(f'Time {datetime.now()}: Accuracy is {accuracy} at round {round} after attack')

if accuracy > initial_accuracy:
    mu *= alpha  # Increase perturbation magnitude
else:
    mu *= beta  # Decrease perturbation magnitude



In [ ]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from copy import deepcopy


# Using SVM for comparison
svm = SVC(C=1, kernel='linear')
svm.fit(Xlocal, ylocal)
w_svm = svm.coef_[0]
b_svm = svm.intercept_[0]

print(f'SVM Coefficients: {w_svm}')
print(f'SVM Intercept: {b_svm}')

# Using SGDClassifier
slc = len(ylocal)
C = 1
alpha = 1 / (slc * C)
sgd = SGDClassifier(loss='hinge', alpha=alpha, max_iter=1000, tol=1e-3, random_state=42)

# Initialize with the same coefficients and intercept as SVM
sgd.coef_ = np.array([deepcopy(w_svm)])  # Set the modified coefficients
sgd.intercept_ = np.array([deepcopy(b_svm)])  # Set the modified intercept

# Train with partial_fit
sgd.partial_fit(Xlocal, ylocal, classes=np.unique(ylocal))

# Extract the coefficients and intercept after training
b_sgd = deepcopy(sgd.intercept_[0])  # Intercept
w_sgd = deepcopy(sgd.coef_[0])  # Coefficients

print(f'SGD Coefficients after partial_fit: {w_sgd}')
print(f'SGD Intercept after partial_fit: {b_sgd}')

# Evaluation function (replace with your actual evaluation method)
def evaluate_accuracy(model, X, y):
    predictions = model.predict(X)
    accuracy = np.mean(predictions == y)
    return accuracy

# Evaluate the SVM model
svm_accuracy = evaluate_accuracy(svm, testX, testy)
print(f'SVM Accuracy: {svm_accuracy * 100:.2f}%')

# Evaluate the SGD model
sgd_accuracy = evaluate_accuracy(sgd, testX, testy)
print(f'SGD Accuracy: {sgd_accuracy * 100:.2f}%')

In [ ]:

#for fed-avg
# Assuming dataLoad() function initializes XGlobal, yGlobal, testX, testy, slc
# dataLoad()  # Ensure this function is defined

num_local_machine = 100
advNumber = 30
dataset_type = 'ACOUSTIC'
MACHINE_LEARNING_ALGO = 'PERCEPTRON' # 'SVM' #'LR'
ATTACK_TYPE='BEAT2FED'
MODE='INDIV'
AGGREGATION = 'MEAN' # 'KRUM'
iid_deg=50
poisoning_percent=100
dataLoad()

# Function to evaluate accuracy
def evaluate_accuracy(tW, tb):
    cnt = 0
    for i in range(len(testX)):
        det = np.dot(testX[i], tW) + tb
        if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
            cnt += 1
    accuracy = cnt / len(testX)

    params = {
        'num_local_machine': num_local_machine,
        'advNumber': advNumber,
        'dataset_type': dataset_type,
        'MACHINE_LEARNING_ALGO': MACHINE_LEARNING_ALGO,
        'ATTACK_TYPE': ATTACK_TYPE,
        'MODE': MODE,
        'AGGREGATION': AGGREGATION,
        'iid_deg': iid_deg,
        'poisoning_percent': poisoning_percent,
        'round': round
    }
    store_results(params, accuracy)

    return accuracy

def average_euclidean_distance(global_models):
    num_models = len(global_models)
    # mu = 0

    # # Compute pairwise distances
    # for i in range(num_models):
    #     for j in range(i + 1, num_models):
    #         mu = max(mu, np.linalg.norm(global_models[i] - global_models[j]))

    # Calculate the average distance

    # Convert list of ndarrays to a 2D array for pdist
    global_models_array = np.vstack(global_models)

    # Compute pairwise Euclidean distances and calculate the standard deviation
    pairwise_distances = pdist(global_models_array, metric='euclidean')
    mu = 3*np.std(pairwise_distances)

    return mu


def multi_krum(W_1, B):
    # Krum
    hSets = deepcopy(W_1)
    num_local_machine = len(W_1)

    krum_scores = []
    byzantine_client_num = num_local_machine // 5
    krum_limit = num_local_machine - byzantine_client_num - 2

    for i in range(num_local_machine):
        dists = []
        for j in range(num_local_machine):
            if i != j:
                temp_vector = np.subtract(hSets[i], hSets[j])
                distance = np.dot(temp_vector, temp_vector)
                dists.append(distance)
        dists.sort()  # ascending
        score = dists[:krum_limit]
        krum_scores.append(sum(score))

    score_index = np.argsort(krum_scores)[:krum_limit]  # indices; ascending

    print(f'Out of {len(score_index)}, {np.sum(score_index < advNumber)} are adversarial')

    hSets = [W_1[i] for i in score_index]
    b = np.mean(B)
    w = np.mean(hSets, axis=0)

    return w, b


def aggregation(W, B):
    if AGGREGATION == 'MEAN':
        return np.mean(W, axis=0), np.mean(B)
    elif AGGREGATION == 'KRUM':
        return multi_krum(W, B)



# Initialize lists to store weights and biases
global_models = []

global_w, global_b = np.zeros(len(XGlobal[0])), 0
prev_global_w = global_w

# Store initial global model
# global_models.append(global_w)


initial_accuracy = evaluate_accuracy(global_w, global_b)
print(f'Time {datetime.now()}: Accuracy is {initial_accuracy} before attack')

# Multi-round federated learning with attack
alpha = 1.1  # Scaling factor for increasing mu
beta = 0.9  # Scaling factor for decreasing mu

for round in range(100):
    W_1, B_1 = [], []
    for local_machine in range(num_local_machine):
        Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
        ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])
        # sgd = SGDClassifier(loss='log', max_iter=100, tol=1e-3, learning_rate='constant', eta0=0.01)
        if MACHINE_LEARNING_ALGO == 'SVM':
            C = 1
            alpha = 1 / (slc * C)
            # SGD Classifier configured as SVM
            sgd = SGDClassifier(loss='hinge', alpha=alpha, max_iter=1000, tol=1e-3, random_state=42)
        elif MACHINE_LEARNING_ALGO == 'LR':
            sgd = SGDClassifier(loss='log', max_iter=20 * len(testy), tol=1e-4, learning_rate='optimal', random_state=42)
        elif MACHINE_LEARNING_ALGO == 'PERCEPTRON':
            sgd = SGDClassifier(loss='perceptron', max_iter=1000, tol=1e-3, random_state=42)
        elif MACHINE_LEARNING_ALGO == 'LDA':
            # Linear Discriminant Analysis equivalent
            sgd = LinearDiscriminantAnalysis()
            sgd.fit(Xlocal, ylocal)




        sgd.coef_ = np.array([deepcopy(global_w)])  # Set the modified coefficients
        sgd.intercept_ = np.array([deepcopy(global_b)])  # Set the modified intercept
        sgd.partial_fit(Xlocal, ylocal, classes=np.unique(ylocal))

        b = deepcopy(sgd.intercept_[0])  # Intercept
        w = deepcopy(sgd.coef_[0])  # Coefficients

        W_1.append(w)
        B_1.append(b)
        # Step 2: Make predictions on the test data
        predictions = sgd.predict(testX)

        # # Step 3: Calculate the accuracy
        # accuracy = accuracy_score(testy, predictions)
        # print(f'For local machine, {local_machine}, Original Accuracy is {accuracy}')


    for local_machine in range(advNumber):
        Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
        ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])

        if round == 0:
            w = W_1[local_machine]
            b = B_1[local_machine]
        else:
            w = global_w
            b = global_b

        v = np.zeros(len(Xlocal[0]))
        for j in range(len(Xlocal)):
            v += np.array(Xlocal[j]) * (-1 if ylocal[j] == 0 else 1)

        # # Min max attack
        # for indx in range(len(v)):
        #     w[indx] = -fabs(w[indx]) if v[indx] > 0 else fabs(w[indx])
            # w[indx] = -1000 if v[indx] > 0 else 1000

        unit_vector = v/np.sqrt(np.dot(v,v))
        # Calculate standard deviation of stored global models
        if len(global_models) > 1:
            mu = max(np.linalg.norm(W_1[local_machine]-global_w), average_euclidean_distance(global_models))
        elif len(global_models) == 1:
            mu = np.linalg.norm(W_1[local_machine]-global_w)
        else:
            # mu = np.linalg.norm(W_1[local_machine]-global_w)
            #mu = 0.1  # Initial perturbation magnitude if only one global model exists
            mu = 1
        # print(f"For round {round} and advNumber {local_machine} mu is {mu}", end=" ")
        delta = mu * unit_vector



        # accuracy = evaluate_accuracy(w, global_b)
        # print(f'Non attacker: Accuracy is {accuracy}', end=" ")

        # # Apply perturbation to the local model
        w = global_w - delta
        # w = w - delta
        # b = b
        # accuracy = evaluate_accuracy(w, b)
        # print(f'Attacker: Accuracy is {accuracy}')
        W_1[local_machine] = w
        B_1[local_machine] = b

    # for local_machine in range(num_local_machine):
    #     accuracy = evaluate_accuracy(W_1[local_machine], B_1[local_machine])
    #     print(f'For local machine {local_machine}, Accuracy is {accuracy}')

    # Update global model
    prev_global_w = global_w
    global_w , global_b = aggregation(W_1, B_1)

    # Store the new global model
    global_models.append(global_w)

    # Evaluate accuracy after attack
    accuracy = evaluate_accuracy(global_w, global_b)
    print(f'Time {datetime.now()}: Accuracy is {accuracy} at round {round} after attack')




In [ ]:
num_local_machine = 100
dataset_type= 'FASHION_MNIST' #'CIFAR10' # 'ACOUSTIC'#'MNIST' ,  ,, , 'IJCNN1', 'madelon'
MACHINE_LEARNING_ALGO= 'LR' #'LDA' , 'PERCEPTRON', , 'SVM' ,
dataLoad()
advNumber = 20

W, B = [], []
W_1, B_1 = [], []

for local_machine in range(num_local_machine):
    Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
    ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])

    if MACHINE_LEARNING_ALGO == 'SVM':
        clfLocals = CClassifierSVM(C=1)
        clfLocals.fit(Xlocal, ylocal)
        w, b = clfLocals.w.get_data()[0], clfLocals.b.get_data()[0]
        W.append(w)
        B.append(b)

    elif MACHINE_LEARNING_ALGO == 'LR':
        model = LogisticRegression(max_iter=20*len(ylocal))

        # Step 4: Train the model
        model.fit(Xlocal, ylocal)

        # Step 5: Get the coefficients from the trained model
        b = model.intercept_[0]  # Intercept
        w = model.coef_[0]
        W.append(w)
        B.append(b)

    elif MACHINE_LEARNING_ALGO == 'PERCEPTRON':
        model = Perceptron()

        # Step 4: Train the model
        model.fit(Xlocal, ylocal)

        # Step 5: Get the coefficients from the trained model
        b = model.intercept_[0]     # Intercept
        w = model.coef_[0]          # Coefficients for each feature
        W.append(w)
        B.append(b)

    elif MACHINE_LEARNING_ALGO == 'LDA':
        model = LinearDiscriminantAnalysis()

        # Step 4: Train the model
        model.fit(Xlocal, ylocal)

        # Step 5: Get the coefficients from the trained model
        b = model.intercept_[0]  # Intercept
        w = model.coef_[0]
        W.append(w)
        B.append(b)

#prev_global_w = global_w = np.mean(W, axis=0)
#global_w = np.mean(W, axis=0)
#global_b = np.mean(B)
# global_w, global_b = multi_krum (W, B)
global_w, global_b = np.zeros(len(XGlobal[0])), 0
prev_global_w = global_w

cnt = 0
for i in range(len(testX)):
    det = np.dot(testX[i], global_w) + global_b
    if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
        cnt += 1
print(f'Time {datetime.now()}: Accuracy is {cnt/len(testX)} before attack')



for round in range(100):
    W_1, B_1 = [], []
    for local_machine in range(num_local_machine):
        Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
        ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])
        sgd = SGDClassifier(loss='log', max_iter=100, tol=1e-3, learning_rate='constant', eta0=0.01)
        sgd.coef_ = np.array([deepcopy(global_w)])  # Set the modified coefficients
        sgd.intercept_ = np.array([deepcopy(global_b)])  # Set the modified intercept
        sgd.partial_fit(Xlocal, ylocal, classes=np.unique(ylocal))

        # Step 5: Get the coefficients from the trained model
        b = deepcopy(sgd.intercept_[0])  # Intercept
        w = deepcopy(sgd.coef_[0])

        # b = deepcopy(global_b)
        # w = deepcopy(global_w)

        W_1.append(w)
        B_1.append(b)

    mu = 0
    for w1 in W_1:
        for w2 in W_1:
            mu = max(mu, np.linalg.norm(w1-w2))
    print(f'mu is {mu}')

    for local_machine in range(num_local_machine):
        if local_machine < advNumber:
            Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
            ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])
            # Do The Attack
            v = np.zeros(len(Xlocal[0]))
            for j in range(len(Xlocal)):
                v = v + np.array(Xlocal[j]) * (-1 if ylocal[j] == 0 else 1)
            unit_vector = v/np.sqrt(np.dot(v,v))
            # unit_vector = global_w/np.sqrt(np.dot(global_w,global_w))
            mu= np.linalg.norm(global_w-prev_global_w)/ np.linalg.norm(v)
            # mu= 2
            # mu= 20 * np.linalg.norm(global_w-prev_global_w)
            #mu= np.var(Xlocal, axis=0, ddof=1)* np.linalg.norm(global_w-w)
            #tmu = max(np.linalg.norm(global_w-W_1[local_machine]), np.linalg.norm(global_w-prev_global_w))
            print(f'for local machine {local_machine} mu is {mu}')
            # print(np.linalg.norm(global_w-W_1[local_machine]), np.linalg.norm(global_w-prev_global_w))
            delta = mu*unit_vector
            # print(local_machine)
            # print(v)
            # print(unit_vector)
            # print(delta)
            w = global_w + delta
            b = global_b
            W_1[local_machine] = w
            B_1[local_machine] = b
            # for i in range(len(v)):
            #     W_1[local_machine][i] = -global_w[i] if v[i] > 0 else global_w[i]

    prev_global_w = global_w
    global_w = np.mean(W_1, axis=0)
    global_b = np.mean(B_1)
    # global_w, global_b = multi_krum(W_1, B_1)

    cnt = 0
    for i in range(len(testX)):
        det = np.dot(testX[i], global_w) + global_b
        if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
            cnt += 1
    print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} at round {round} after attack')

In [ ]:
num_local_machine = 100
dataset_type= 'MNIST' # 'FASHION_MNIST' #'CIFAR10' #,  ,, 'ACOUSTIC', 'IJCNN1', 'madelon'
MACHINE_LEARNING_ALGO= 'SVM' # 'LR' #'LDA' , 'PERCEPTRON', , ,
dataLoad()
advNumber = 0

W, B = [], []
W_1, B_1 = [], []

for local_machine in range(num_local_machine):
    Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
    ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])

    if MACHINE_LEARNING_ALGO == 'SVM':
        # model = SVC(C=1, kernel='linear')

        # # Step 4: Train the model
        # model.fit(Xlocal, ylocal)


        C = 1
        alpha = 1 / (slc * C)
        model = SGDClassifier(loss='hinge', alpha=alpha, max_iter=1000, tol=1e-3, random_state=42)

        model.partial_fit(Xlocal, ylocal)

        # Step 5: Get the coefficients from the trained model
        b = model.intercept_[0]  # Intercept
        w = model.coef_[0]
        W.append(w)
        B.append(b)

    elif MACHINE_LEARNING_ALGO == 'LR':
        model = LogisticRegression(max_iter=20*len(ylocal))

        # Step 4: Train the model
        model.fit(Xlocal, ylocal)

        # Step 5: Get the coefficients from the trained model
        b = model.intercept_[0]  # Intercept
        w = model.coef_[0]
        W.append(w)
        B.append(b)

    elif MACHINE_LEARNING_ALGO == 'PERCEPTRON':
        model = Perceptron()

        # Step 4: Train the model
        model.fit(Xlocal, ylocal)

        # Step 5: Get the coefficients from the trained model
        b = model.intercept_[0]     # Intercept
        w = model.coef_[0]          # Coefficients for each feature
        W.append(w)
        B.append(b)

    elif MACHINE_LEARNING_ALGO == 'LDA':
        model = LinearDiscriminantAnalysis()

        # Step 4: Train the model
        model.fit(Xlocal, ylocal)

        # Step 5: Get the coefficients from the trained model
        b = model.intercept_[0]  # Intercept
        w = model.coef_[0]
        W.append(w)
        B.append(b)

global_w = np.mean(W, axis=0)
global_b = np.mean(B)


cnt = 0
for i in range(len(testX)):
    det = np.dot(testX[i], global_w) + global_b
    if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
        cnt += 1
print(f'Time {datetime.now()}: Accuracy is {cnt/len(testX)} before attack')

for local_machine in range(num_local_machine):
    Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
    ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])
    if local_machine < advNumber:
        # Do The Attack
        v = np.zeros(len(Xlocal[0]))
        for j in range(len(testX)):
            v = v + np.array(testX[j]) * (-1 if testy[j] == 0 else 1)
        unit_vector = v/np.sqrt(np.dot(v,v))
        mu= np.linalg.norm(global_w-W[local_machine])
        print(f'for local machine {local_machine} mu is {mu}')
        delta = -mu*unit_vector
        # print(local_machine)
        # print(v)
        # print(unit_vector)
        # print(delta)
        W[local_machine] = global_w + delta
    W_1.append(W[local_machine])

w = np.mean(W_1, axis=0)
b = global_b

cnt = 0
for i in range(len(testX)):
    det = np.dot(testX[i], w) + b
    print(det, testy[i])
    if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
        cnt += 1
print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} after average')


File stored in `/root/secml-data/datasets/mnist/train-images-idx3-ubyte.gz`

File stored in `/root/secml-data/datasets/mnist/train-labels-idx1-ubyte.gz`

File stored in `/root/secml-data/datasets/mnist/t10k-images-idx3-ubyte.gz`

File stored in `/root/secml-data/datasets/mnist/t10k-labels-idx1-ubyte.gz`


Loading MNIST training dataset from /root/secml-data/datasets/mnist...

Loading MNIST testing dataset from /root/secml-data/datasets/mnist...

ValueError: classes must be passed on the first call to partial_fit.